In [4]:
from __future__ import print_function
import torch

In [5]:
#uninitialized 5x3 matrix
x = torch.empty(5,3)
print(x)
#randomly initialized matrix
y = torch.rand(5,3)
print(y)
#matrix filled with zeros type long
z = torch.zeros(5, 3, dtype=torch.long)
print(z)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.8049e-42,  0.0000e+00],
        [ 0.0000e+00,  4.2621e+31,  0.0000e+00]])
tensor([[ 0.0348,  0.4634,  0.9655],
        [ 0.0179,  0.0390,  0.2067],
        [ 0.0345,  0.6940,  0.2548],
        [ 0.9100,  0.7103,  0.4060],
        [ 0.9760,  0.7475,  0.4781]])
tensor([[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]])


In [6]:
#tenosr from a data 
x = torch.tensor([5.5, 3])
print(x)

tensor([ 5.5000,  3.0000])


In [7]:
#construct tensor based on another one
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)
print(x.size())

tensor([[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]], dtype=torch.float64)
tensor([[ 0.3657,  0.7377, -1.0102],
        [-0.7760,  0.3917, -0.0214],
        [-1.2992,  0.9069, -1.1739],
        [-0.5347,  0.2248, -0.9063],
        [ 0.4824, -0.2638,  0.6159]])
torch.Size([5, 3])


In [8]:
x = torch.ones(5)
print(x)

tensor([ 1.,  1.,  1.,  1.,  1.])


In [9]:
y = x.numpy()
print(y.dtype)

float32


In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double)) 
else:
    print("No GPU")

No GPU


In [11]:
#Autograd tuts
x= torch.ones(2,2,requires_grad=True)
print(x)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


In [12]:
#do an op on x and chek grad_fn 
y = x +2
print(y)
print(y.grad_fn)

tensor([[ 3.,  3.],
        [ 3.,  3.]])


In [23]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_()
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [24]:
z = y * y * 3
out = z.mean()

print(z, out)
out.backward()
print(x.grad)

tensor([[ 27.,  27.],
        [ 27.,  27.]]) tensor(27.)
tensor([[ 22.5000,  22.5000],
        [ 22.5000,  22.5000]])


In [29]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)
print(x)

tensor([-534.9380, -896.8801,  280.0901])
tensor([-1.0448, -1.7517,  0.5471])


In [30]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([  51.2000,  512.0000,    0.0512])


In [31]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


In [32]:
#basic neural net model 
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [35]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [36]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0883,  0.0054, -0.0573, -0.0844,  0.0578,  0.0109,  0.0704,
         -0.1049,  0.0664,  0.0075]])


In [37]:
#zeroing gradient bufferes before propogating
net.zero_grad()
out.backward(torch.randn(1, 10))

In [38]:
#adding loss function 
output = net(input)
target = torch.arange(1, 11)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(38.4858)


In [39]:
#checking previous steps
'''input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss'''
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  #

In [40]:
#backpropogate the error to nn again
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor([ 0.0750,  0.0359,  0.1016, -0.0136,  0.0650, -0.1036])


In [42]:
#update the weights
#weight = weight - learning_rate * gradient
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update